In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/darija-reviews1/darija_reviews_cleaned.csv


In [3]:
data = pd.read_csv("/kaggle/input/darija-reviews1/darija_reviews_cleaned.csv")

In [4]:
df = pd.DataFrame(data)

In [5]:
df

,cleaned_review,rating,sentiment
0,attention smart mais pas android,4.0,positive
1,excellent qualite prix,5.0,positive
2,mzyanz,5.0,positive
3,koooooooooooooo,1.0,negative
4,ma 3jebatnich,1.0,negative
...,...,...,...
2777,pratique et cool,5.0,positive
2778,shy,5.0,positive
2779,c bon rapport qualite priix mrc,3.0,positive
2780,c est bon,4.0,positive


In [6]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Conversion en chaîne de caractères
df['cleaned_review'] = df['cleaned_review'].astype(str)

# Tokenization
tokenized_reviews = [word_tokenize(review) for review in df['cleaned_review']]

# Entraînement du modèle Word2Vec
model_word2vec = Word2Vec(sentences=tokenized_reviews, vector_size=100, window=5, min_count=1, workers=4)

# Sauvegarde du modèle
model_word2vec.save("word2vec_model.model")


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np
import nltk
nltk.download('punkt')

# Charger le modèle Word2Vec
model_word2vec = Word2Vec.load("word2vec_model.model")

# Tokenization
tokenized_reviews = [word_tokenize(review) for review in df['cleaned_review']]

# Créer des embeddings pour chaque critique en moyennant les embeddings des mots
review_embeddings = [
    np.mean([model_word2vec.wv[word] for word in review if word in model_word2vec.wv], axis=0)
    if any(word in model_word2vec.wv for word in review)
    else np.zeros(model_word2vec.vector_size)  # Vecteur de zéros si aucun mot n'est présent
    for review in tokenized_reviews
]

# Transformation en array pour les données X
X = np.array(review_embeddings)

# Labels
y = df['sentiment']

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
# Modèle de classification (exemple avec régression logistique)


from sklearn.svm import LinearSVC

clf = LinearSVC()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)



/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

0.8994614003590664

In [9]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


LogisticRegression()

In [10]:
# Prédiction sur l'ensemble de test
y_pred = clf.predict(X_test)

# Calcul de l'accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy du modèle de classification : {accuracy}")

Accuracy du modèle de classification : 0.8994614003590664


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

In [11]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

0.9102333931777379

In [16]:
from nltk.tokenize import word_tokenize
import numpy as np

# Liste de nouvelles critiques (à remplacer par vos propres textes)
new_reviews = [
    "C'est un excellent produit, je suis très satisfait!",
    "koooooooooooooo.",
    "Produit moyen, pourrait être amélioré.",
    "Très bon rapport qualité-prix.",
    "ma 3jebatnich."
]

# Pré-traitement des nouvelles critiques
new_reviews_tokens = [word_tokenize(review.lower()) for review in new_reviews]

# Création des embeddings pour les nouvelles critiques
new_reviews_embeddings = [
    np.mean([model_word2vec.wv[word] for word in review if word in model_word2vec.wv], axis=0)
    if any(word in model_word2vec.wv for word in review)
    else np.zeros(model_word2vec.vector_size)
    for review in new_reviews_tokens
]

# Transformation en array pour les données X
new_reviews_array = np.array(new_reviews_embeddings)

# Faire les prédictions avec le modèle RandomForest
predictions = clf.predict(new_reviews_array)

# Afficher les prédictions pour chaque nouvelle critique
for i, review in enumerate(new_reviews):
    print(f"Review: {review}")
    print(f"Sentiment prédit: {predictions[i]}")
    print()


Review: C'est un excellent produit, je suis très satisfait!
Sentiment prédit: positive

Review: koooooooooooooo.
Sentiment prédit: negative

Review: Produit moyen, pourrait être amélioré.
Sentiment prédit: positive

Review: Très bon rapport qualité-prix.
Sentiment prédit: positive

Review: ma 3jebatnich.
Sentiment prédit: negative

